# Process Algebra 2010 Dataset

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from dateutil import parser
from utils import min_seq_len_filter

In [2]:
MIN_SEQ_LENGTH = 10

In [3]:
path = "./raw-datasets/All_Data_1872_2013_0128_181549.txt"
kc_col = "KC(KTracedSkills)"
df = pd.read_csv(path, sep='\t')
ix = ~df[kc_col].isnull()
df = df[ix]    

/home/mmkhajah/venvs/sci/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Predicted Error Rate(KTracedSkills),KC(Rules),Opportunity(Rules),Predicted Error Rate(Rules),KC(Single-KC),Opportunity(Single-KC),Predicted Error Rate(Single-KC),KC(Unique-step),Opportunity(Unique-step),Predicted Error Rate(Unique-step)
5,6,stu_00031c,"Unit CTA1_06, Section CTA1_06-4",BH1T51,1,R3C2,2011-01-31 02:41:39.0,2011-01-31 02:42:13.0,2011-01-31 02:42:41.0,2011-01-31 02:42:41.0,...,NaN,COUNT-NOUN-MX+B-FORMULA-HELP,1,NaN,Single-KC,6.0,NaN,KC1240,1.0,NaN
6,7,stu_00031c,"Unit CTA1_06, Section CTA1_06-4",BH1T51,1,R4C2,2011-01-31 02:42:41.0,2011-01-31 02:42:55.0,2011-01-31 02:42:59.0,2011-01-31 02:42:59.0,...,NaN,GIVEN-HELP-DONT-REPEAT-UNIT,1,NaN,Single-KC,7.0,NaN,KC580,1.0,NaN
9,10,stu_00031c,"Unit CTA1_06, Section CTA1_06-4",BH1T51,1,R4C1,2011-01-31 02:43:23.0,2011-01-31 02:43:25.0,2011-01-31 02:43:29.0,2011-01-31 02:43:29.0,...,NaN,CALCULATED-VALUE-GIVEN-Y-EXPRESSION-COMPLETED,1,NaN,Single-KC,10.0,NaN,KC2395,1.0,NaN
10,11,stu_00031c,"Unit CTA1_06, Section CTA1_06-4",BH1T51,1,R5C2,2011-01-31 02:43:29.0,2011-01-31 02:43:42.0,2011-01-31 02:43:54.0,2011-01-31 02:43:54.0,...,NaN,GIVEN-HELP-DONT-REPEAT-UNIT,2,NaN,Single-KC,11.0,NaN,KC1975,1.0,NaN
13,14,stu_00031c,"Unit CTA1_06, Section CTA1_06-4",BH1T51,1,R5C1,2011-01-31 02:44:17.0,2011-01-31 02:44:22.0,2011-01-31 02:44:22.0,2011-01-31 02:44:22.0,...,NaN,CALCULATED-VALUE-GIVEN-Y-EXPRESSION-COMPLETED,2,NaN,Single-KC,14.0,NaN,KC2750,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3178588,3178589,stu_fffba2,"Unit CTA1_01, Section CTA1_01-2",BH1T08A,1,R5C2,2010-08-26 17:50:07.0,2010-08-26 17:50:10.0,2010-08-26 17:50:10.0,2010-08-26 17:50:10.0,...,NaN,GIVEN-HELP-NONE,6,NaN,Single-KC,93.0,NaN,NaN,NaN,NaN
3178589,3178590,stu_fffba2,"Unit CTA1_01, Section CTA1_01-2",BH1T08A,1,R5C1,2010-08-26 17:49:26.0,2010-08-26 17:49:37.0,NaN,2010-08-26 17:51:45.0,...,NaN,WRONG-COLUMN-FOR-GIVEN~~CALCULATED-VALUE-GIVEN-Y,1~~1,NaN,Single-KC,94.0,NaN,NaN,NaN,NaN
3178590,3178591,stu_fffba2,"Unit CTA1_01, Section CTA1_01-2",BH1T08A,1,R6C1,2010-08-26 17:51:45.0,2010-08-26 17:51:52.0,2010-08-26 17:51:52.0,2010-08-26 17:51:52.0,...,NaN,GIVEN-HELP-NONE,7,NaN,Single-KC,95.0,NaN,NaN,NaN,NaN
3178591,3178592,stu_fffba2,"Unit CTA1_01, Section CTA1_01-2",BH1T08A,1,R6C2,2010-08-26 17:51:52.0,2010-08-26 17:51:55.0,2010-08-26 17:52:02.0,2010-08-26 17:52:02.0,...,NaN,CALCULATED-VALUE-GIVEN-X,19,NaN,Single-KC,96.0,NaN,NaN,NaN,NaN


In [5]:
df['corr_dur_sec'] = df['Correct Step Duration (sec)'].fillna(0).astype(float)
df['error_dur_sec'] = df['Error Step Duration (sec)'].replace(0).astype(float)
df['problem'] = df['Problem Hierarchy'].astype(str) + '>>' + df['Problem Name'].astype(str) + '>>' + df['Step Name'].astype(str)

In [6]:
def make_indecies(series):
    unique_vals = sorted(set(series))
    return dict(zip(unique_vals, range(len(unique_vals))))
def make_mapping(keys_series, vals_series):
    mapping = defaultdict(set)
    for k, v in zip(keys_series, vals_series):
        mapping[k].add(v)
    return dict(mapping)

In [7]:
problem_id = make_indecies(df['problem'])
student_id = make_indecies(df['Anon Student Id'])
skill_id = make_indecies(df[kc_col])
df['end_time'] = pd.to_datetime(df['Step End Time'])
df['student'] = df['Anon Student Id']
df['correct'] = (df['First Attempt'] == 'correct').astype(int)
df['response_time_ms'] = (df['correct'] * df['corr_dur_sec'] + (1-df['correct']) * df['error_dur_sec']) * 1000
df['skill'] = df[kc_col]

In [8]:
rows = []
for r in df.itertuples():
    rows.append({
        "student" : student_id[r.student],
        "skill" : skill_id[r.skill],
        "correct" : r.correct,
        "end_time" : r.end_time,
        "problem" : problem_id[ r.problem ],
        "response_time_ms" : r.response_time_ms,
        "skill_name" : r.skill
    })
output_df = pd.DataFrame(rows)
output_df = min_seq_len_filter(output_df, MIN_SEQ_LENGTH)

In [9]:
print("Trials: %d" % output_df.shape[0])
print("Students: %d" % len(set(output_df['student'])))
print("Skills: %d" % len(set(output_df['skill'])))
print("Problems: %d" % len(set(output_df['problem'])))
date = pd.to_datetime(output_df['end_time'])
min_time = np.min(date)
max_time = np.max(date)
print("Time range: %s to %s" % (min_time, max_time))

Trials: 1562060
Students: 3291
Skills: 61
Problems: 4116
Time range: 2010-08-17 13:16:28 to 2018-03-31 03:23:59


In [10]:
output_df.to_csv("datasets/algebra2010.csv", index=False)